## OpenCV: PCA

fonte: [https://docs.opencv.org/3.4/da/d60/tutorial_face_main.html](https://docs.opencv.org/3.4/da/d60/tutorial_face_main.html)

### Preparando ambiente

In [4]:
!pip install --upgrade face_recognition
!pip install --upgrade opencv-python

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Requirement already up-to-date: face_recognition in /usr/local/lib/python3.7/site-packages (1.3.0)
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Requirement already up-to-date: opencv-python in /usr/local/lib/python3.7/site-packages (4.5.1.48)


### Importando bibliotecas e definindo funções

In [5]:
import face_recognition
import cv2
import numpy as np

from IPython.display import display, Javascript, Image, clear_output
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
import urllib.request

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  resp = urllib.request.urlopen(data)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)

  return image

ModuleNotFoundError: No module named 'google.colab'

### Importando faces conhecidas
download das fotos

In [ ]:
!wget -O humberto.jpg 'https://media-exp1.licdn.com/dms/image/C4D03AQFkFrCH3eVxQw/profile-displayphoto-shrink_100_100/0/1616287577028?e=1623283200&v=beta&t=49wqUyWCR36zzmkdBAbRs4ey_VRJzTH0XgbhdKxbflw'
!ls -la

### Carregando imagens para serem reconhecidas

In [ ]:
# Load a sample picture and learn how to recognize it.
humberto_image = face_recognition.load_image_file("humberto.jpg")
humberto_face_encoding = face_recognition.face_encodings(humberto_image)[0]

# Create arrays of known face encodings and their names
known_face_encodings = [
    humberto_face_encoding
]
known_face_names = [
    "Humberto"
]

### Reconhecendo faces

In [ ]:
# Inicializa as variáveis de resultados
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
first = True

filename = 'result.jpg'
scale = 4

while True:
    # Captura um simples frame (bate uma foto)
    frame = take_photo()

    # Reduz a dimensão do frame para 1/4 para acelerar o processo de reconhecimento
    small_frame = cv2.resize(frame, (0, 0), fx=1/scale, fy=1/scale)

    # Converte o frame de BGR (OpenCV usa) para RGB (face_recognition usa)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Procura todas as faces no frame atual
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # Procura a face na lista de faces conhecidas
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # Dentre as escolhidas, encontrar a face com a menor distância
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame

    # Exibe os resultados
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Rescala a image para o tamanho original
        top *= scale
        right *= scale
        bottom *= scale
        left *= scale

        # Desenha um retângulo em torno da face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Desenha um rótulo para a face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (255, 0, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        
    clear_output()
    # Exibindo o resultado
    cv2_imshow(frame)
